In [1]:
import pandas as pd
import numpy as np
from Merge import *
from sklearn.cross_validation import train_test_split
from model_builder import *
from itertools import combinations

/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#load data
train= pd.read_csv('imbalanced_data.csv')

#test= pd.read_csv('test_file_1.csv') 



/Users/linda/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.loc[~train.time_of_last_delivery_any_installed_app.isnull(),'time_of_last_delivery_any_installed_app']='t'
train.time_of_last_delivery_any_installed_app=train.time_of_last_delivery_any_installed_app.fillna('f')

In [4]:
#join table
iso = 'metadata/ios_apps_metadata000'
android = 'metadata/android_apps_metadata000'
application = 'metadata/applications000'
meta_tuple= (iso,android,application)
creative_tuple = ('metadata/creatives000','metadata/creative_tags000',\
                 'metadata/video_files000')
iso=['market_id','developer','created_at','last_updated_at','content_rating',
    'price_currency_code','supported_devices',
    'user_rating','n_user_ratings'] 
andriod =['market_id','developer','created_at','last_updated_at','content_rating','primary_category',
          'price_currency_code','user_rating','n_user_ratings',
          'has_in_app_purchases','n_min_downloads','n_max_downloads']
meta_column=(iso,andriod)
creat_col =['delivery_frequency_cap','is_streaming','cloudux_template_id',
            'enforce_language_match','time_to_show_countdown','time_to_show_close_button']

yX_train =combine_meta_data(train,meta_tuple,creative_tuple,meta_column,creat_col,option='both')

In [10]:
#yX_train['time_delta_updated'] = pd.to_datetime(yX_train['last_updated_at'],errors='coerce')-pd.to_datetime(yX_train['created_at'],errors='coerce')
#yX_train['time_delta_updated'] = yX_train['time_delta_updated'].dt.days


In [9]:
#yX_train = yX_train[yX_train.n_vungle_installs.apply(lambda x:isinstance(x, (int,float)))]
#yX_train.n_vungle_installs=yX_train.n_vungle_installs.astype(float)
#yX_train.n_campaign_views=yX_train.n_campaign_views.astype(float)


In [5]:
random_list = [u'campaign_id',u'city',u'time_zone', u'device_language', u'device_make', u'device_model',
               u'platform', u'developer',u'content_rating','user_rating','n_user_ratings',
             u'price_currency_code',u'supported_devices','n_min_downloads','n_max_downloads',
            'enforce_language_match','time_to_show_countdown','time_to_show_close_button']

medium_list =[ u'device_platform', u'device_connection', u'time_of_last_delivery_this_campaign',
             u'delivery_frequency_cap', u'is_streaming', u'cloudux_template_id',u'is_publisher',u'device_os_version',
             u'primary_category']

good_list = [u'n_campaign_views', u'n_vungle_installs']

In [10]:
for i in random_list+medium_list+good_list:
    print i,'----',len(yX_train[i].unique())

campaign_id ---- 4656
city ---- 18491
time_zone ---- 428
device_language ---- 187
device_make ---- 5175
device_model ---- 36503
platform ---- 4
developer ---- 622
content_rating ---- 10
user_rating ---- 10
n_user_ratings ---- 3865
price_currency_code ---- 1528
supported_devices ---- 2760
n_min_downloads ---- 20
n_max_downloads ---- 20
enforce_language_match ---- 3
time_to_show_countdown ---- 4
time_to_show_close_button ---- 6
time_of_last_delivery_this_campaign ---- 1062664
device_platform ---- 5
device_connection ---- 4
device_os_version ---- 251
delivery_frequency_cap ---- 6
is_streaming ---- 3
cloudux_template_id ---- 6
is_publisher ---- 3
primary_category ---- 42
n_campaign_views ---- 814
n_vungle_installs ---- 61


In [6]:
def logistic_model_final(X_train, y_train,c=0.1):
    """
    use crossvalidation (CV) to report the best parameter 'C'
    parameter C: Inverse of regularization strength; must be a positive float. 
    Check LogisticRegression() in sklearn for more information
    """
    print('Train Regression Model')

    model=LogisticRegression(C=c,penalty='l2',solver='sag',n_jobs=-1)

    model.fit(X_train, y_train)
    return model


In [7]:
def model_selection_fun(b_f,mod,interactions):
    
    #cont_features_pc = []
    cat_interactions_pc = interactions
    cat_features_pc =mod
    train, test = train_test_split(b_f, test_size=0.2,random_state= 42)  # splitting Train into Train/Test sets
    X_train, y_train, X_test, y_test = train, train.pop('is_install'), test, test.pop('is_install')

    feature_creator = FeatureCreator()
    design_matrix_transformer = FeatureHasher(22, cat_features_pc, None, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

    X_test = feature_creator.transform(X_test, inplace=True)
    X_test, f_map = design_matrix_transformer.fit_transform(X_test)

    X_train = feature_creator.transform(X_train, inplace=True)
    X_train, f_map = design_matrix_transformer.fit_transform(X_train)

    logistic_baseline = logistic_model_final(X_train, y_train,0.05)

    # Calculate prediction/probability of train and test
    X_train_predictions = logistic_baseline.predict(X_train)
    X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]

    X_test_predictions = logistic_baseline.predict(X_test)
    X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]


    # Calculate metrics of train, validation and test set.
    #lr_ll_val = -logistic_baseline.best_score_

    lr_ll_train = log_loss(y_train, X_train_predprob)
    lr_auc_train = roc_auc_score(y_train, X_train_predprob)

    lr_ll_test = log_loss(y_test, X_test_predprob)    
    lr_auc_test = roc_auc_score(y_test, X_test_predprob)

    # Print out the results
    #print "Best parameter: ", logistic_baseline.best_params_

    #print "Log Loss (Validation): %f" % lr_ll_val

    print "Log Loss (Train): %f" % lr_ll_train
    print "AUC (Train): %f" % lr_auc_train
    #AUC[i]=lr_ll_test
    #Log_lost[i]=lr_auc_test
    print 'Log Loss (Test): %f' % lr_ll_test
    print 'AUC (Test): %f' % lr_auc_test


In [8]:
def logistic_model(X_train, y_train):
    """
    use crossvalidation (CV) to report the best parameter 'C'
    parameter C: Inverse of regularization strength; must be a positive float. 
    Check LogisticRegression() in sklearn for more information
    """
    print('Train Regression Model')
    model = GridSearchCV(
            
            estimator=LogisticRegression(penalty='l2',solver='sag',n_jobs=-1),
            param_grid={'C': [0.05,0.1]},
            scoring='log_loss',
            cv=3
    )
    model.fit(X_train, y_train)
    return model

In [9]:
import random
def choice():
    ran = random.sample(random_list,3)
    median = random.sample(medium_list,2)
    good = good_list
    return ran,median,good,combo(ran),combo(median),combo(good)

def combo(lst): 

    return [i for i in sum([map(tuple, combinations(lst, i)) for i in range(len(lst) + 1)], [])[1:] if len(i) >1]

#### this is the one you are pretty sure it can generate good model you are trying to use it to tune parameter



In [ ]:

base_model = ['advertiser_app_store_id', 'country_code',u'device_language', 'publisher_app_store_id', 
 'n_user_ratings','user_rating',u'n_campaign_views', u'n_vungle_installs',
 u'delivery_frequency_cap','time_to_show_countdown',u'supported_devices',  u'device_platform',
 u'is_streaming']
#### do not use all the interaction terms!!! put them into smaller bins
inter1=['advertiser_app_store_id', 'country_code',u'device_language', 'publisher_app_store_id']
#inter2= ['n_user_ratings', 'user_rating','time_to_show_countdown']
inter3= [ u'n_campaign_views', u'n_vungle_installs', u'delivery_frequency_cap']
#inter4= [u'supported_devices',  u'device_platform']
interaction= combo(inter1) # +combo(inter2)+combo(inter3)+combo(inter4)
yX = yX_train[['is_install']+base_model].copy()
yX.n_user_ratings = yX.n_user_ratings.astype(str).apply(lambda x: len(x))
b = convert_to_unknown(yX,base_model)
b = convert_to_string(b,base_model)
print '------base model-------'
print base_model
print '------interaction-----'
print interaction
model_selection_fun(b,base_model,interaction )

------base model-------
['advertiser_app_store_id', 'country_code', u'device_language', 'publisher_app_store_id', 'n_user_ratings', 'user_rating', u'n_campaign_views', u'n_vungle_installs', u'delivery_frequency_cap', 'time_to_show_countdown', u'supported_devices', u'device_platform', u'is_streaming']
------interaction-----
[('advertiser_app_store_id', 'country_code'), ('advertiser_app_store_id', u'device_language'), ('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', u'n_campaign_views'), ('advertiser_app_store_id', u'n_vungle_installs'), ('advertiser_app_store_id', u'delivery_frequency_cap'), ('country_code', u'device_language'), ('country_code', 'publisher_app_store_id'), ('country_code', u'n_campaign_views'), ('country_code', u'n_vungle_installs'), ('country_code', u'delivery_frequency_cap'), (u'device_language', 'publisher_app_store_id'), (u'device_language', u'n_campaign_views'), (u'device_language', u'n_vungle_installs'), (u'device_language', u'deli

#### Run this when you trying to randomly pick parameters

In [ ]:
sample_interact= [ ('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'),
               ('advertiser_app_store_id', 'publisher_app_store_id'),
               ('advertiser_app_store_id', 'country_code'),('publisher_app_store_id', 'country_code')]
for i in range(20):
    ran,med,good,r,m,g = choice()
    base_model = ['advertiser_app_store_id','country_code','publisher_app_store_id']+ran+med+good
    interactions = combo(base_model)  
    yX = yX_train[['is_install']+base_model].copy()
    b = convert_to_unknown(yX,base_model)
    b = convert_to_string(b,base_model)
    print '------base model-------'
    print base_model
    interaction=random.sample(interactions,len(interactions)//100)
    interaction = interaction+ [i for i in sample_interact if i not in interaction]
    interaction = interaction+ [i for i in r if i not in interaction]
    interaction = interaction+ [i for i in m if i not in interaction]
    interaction = interaction+ [i for i in g if i not in interaction]
    print '------interaction-----'
    print interaction
    model_selection_fun(b,base_model,interaction )


------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'city', u'device_language', 'user_rating', u'delivery_frequency_cap', u'is_streaming', u'n_campaign_views', u'n_vungle_installs']
------interaction-----
[('advertiser_app_store_id', 'publisher_app_store_id', u'city', u'device_language', 'user_rating', u'delivery_frequency_cap', u'is_streaming', u'n_campaign_views', u'n_vungle_installs'), ('publisher_app_store_id', u'city', u'device_language', u'delivery_frequency_cap', u'is_streaming', u'n_vungle_installs'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'city', u'delivery_frequency_cap', u'is_streaming'), ('advertiser_app_store_id', 'publisher_app_store_id', u'city', u'device_language', u'n_campaign_views', u'n_vungle_installs'), (u'device_language', 'user_rating', u'delivery_frequency_cap', u'n_campaign_views', u'n_vungle_installs'), ('publisher_app_store_id', u'city', u'device_language', u'n_campaign_views'), ('publ

In [36]:
#base_model = ['advertiser_app_store_id','country_code','publisher_app_store_id']
#string_list=[[ u'time_of_last_delivery_this_campaign',u'device_make',u'device_model',u'device_os_version',u'n_campaign_views', u'n_vungle_installs']]
for string in string_list:
    base_model = ['advertiser_app_store_id','country_code','publisher_app_store_id']+[string]
    yX = yX_train[['is_install']+base_model].copy()
    b = convert_to_unknown(yX,base_model)
    b = convert_to_string(yX,base_model)
    print base_model
    model_selection_fun(b,base_model,[])


#transaction data
string_list =[ u'device_language', u'device_make', u'device_model',
       u'device_platform', u'device_connection', u'device_os_version', u'event_id', u'n_campaign_views',
              u'n_vungle_installs', u'time_of_last_delivery_this_campaign',
             u'time_zone',u'market_id',u'is_publisher', u'platform',u'is_streaming', u'cloudux_template_id']

['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'time_of_last_delivery_this_campaign', u'device_make', u'device_model', u'device_os_version', u'n_campaign_views', u'n_vungle_installs']  
Train Regression Model   
Log Loss (Train): 0.396761  
AUC (Train): 0.790704  
Log Loss (Test): 0.398367  
AUC (Test): 0.789067  
**['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'n_campaign_views']   
Log Loss (Train): 0.403693  
AUC (Train): 0.780206    
Log Loss (Test): 0.405053  
AUC (Test): 0.778919  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'n_vungle_installs']    
Log Loss (Train): 0.407603  
AUC (Train): 0.774281  
Log Loss (Test): 0.409066  
AUC (Test): 0.772736  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'time_of_last_delivery_this_campaign']   
Log Loss (Train): 0.409217  
AUC (Train): 0.771295  
Log Loss (Test): 0.410794  
AUC (Test): 0.769634 
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_os_version']    
Log Loss (Train): 0.409170  
AUC (Train): 0.771266  
Log Loss (Test): 0.410792  
AUC (Test): 0.769447**  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_language']  
Log Loss (Train): 0.410969  
AUC (Train): 0.768606  
Log Loss (Test): 0.412586  
AUC (Test): 0.766808   
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_make']   
Log Loss (Train): 0.410095  
AUC (Train): 0.769943  
Log Loss (Test): 0.411738  
AUC (Test): 0.768131  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_model']  
Log Loss (Train): 0.409752  
AUC (Train): 0.770506  
Log Loss (Test): 0.411509  
AUC (Test): 0.768511  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_platform']   
Log Loss (Train): 0.411087  
AUC (Train): 0.768389  
Log Loss (Test): 0.412720  
AUC (Test): 0.766588  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_connection']     
Log Loss (Train): 0.411073  
AUC (Train): 0.768410   
Log Loss (Test): 0.412701  
AUC (Test): 0.766619   
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'event_id']   
Log Loss (Train): 0.411265  
AUC (Train): 0.768127  
Log Loss (Test): 0.412959  
AUC (Test): 0.766197     
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'time_zone']    
Log Loss (Train): 0.410875  
AUC (Train): 0.768709  
Log Loss (Test): 0.412487  
AUC (Test): 0.766931  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'market_id']   
Log Loss (Train): 0.410963  
AUC (Train): 0.768581  
Log Loss (Test): 0.412638  
AUC (Test): 0.766701  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'is_publisher']  
Log Loss (Train): 0.411075  
AUC (Train): 0.768409  
Log Loss (Test): 0.412710   
AUC (Test): 0.766604   
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'platform']  
Log Loss (Train): 0.411088  
AUC (Train): 0.768387  
Log Loss (Test): 0.412721  
AUC (Test): 0.766586  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'is_streaming']    
Log Loss (Train): 0.411092  
AUC (Train): 0.768378  
Log Loss (Test): 0.412730  
AUC (Test): 0.766564  
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'cloudux_template_id']    
Log Loss (Train): 0.411072  
AUC (Train): 0.768420  
Log Loss (Test): 0.412706  
AUC (Test): 0.766600  

### Meta data
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'supported_devices']
Train Regression Model
Log Loss (Train): 0.410917
AUC (Train): 0.768549
Log Loss (Test): 0.412878
AUC (Test): 0.765728
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'package_name']
Train Regression Model
Log Loss (Train): 0.410843
AUC (Train): 0.768669
Log Loss (Test): 0.412803
AUC (Test): 0.765849
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'title']
Train Regression Model
Log Loss (Train): 0.410814
AUC (Train): 0.768716
Log Loss (Test): 0.412795
AUC (Test): 0.765875
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'market_id']
Train Regression Model
Log Loss (Train): 0.410791
AUC (Train): 0.768751
Log Loss (Test): 0.412780
AUC (Test): 0.765880
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'price_currency_code']
Train Regression Model
Log Loss (Train): 0.410831
AUC (Train): 0.768663
Log Loss (Test): 0.412764
AUC (Test): 0.765916
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'content_rating']
Train Regression Model
Log Loss (Train): 0.410885
AUC (Train): 0.768594
Log Loss (Test): 0.412847
AUC (Test): 0.765785
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'version']
Train Regression Model
Log Loss (Train): 0.410771
AUC (Train): 0.768797
Log Loss (Test): 0.412738
AUC (Test): 0.765960
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'min_os_version']
Train Regression Model
Log Loss (Train): 0.410907
AUC (Train): 0.768565
Log Loss (Test): 0.412858
AUC (Test): 0.765765
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'has_in_app_purchases']
Train Regression Model
Log Loss (Train): 0.410926
AUC (Train): 0.768527
Log Loss (Test): 0.412896
AUC (Test): 0.765689
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'primary_category']
Train Regression Model
Log Loss (Train): 0.410857
AUC (Train): 0.768612
Log Loss (Test): 0.412797
AUC (Test): 0.765855
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', 'developer']
Train Regression Model
Log Loss (Train): 0.410819
AUC (Train): 0.768702
Log Loss (Test): 0.412757
AUC (Test): 0.765943

------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'has_in_app_purchases', u'device_make', u'time_zone', u'min_os_version', u'version', u'n_vungle_installs', u'time_of_last_delivery_this_campaign']
------interaction-----
Log Loss (Train): 0.403327
AUC (Train): 0.780532
Log Loss (Test): 0.405645
AUC (Test): 0.777235
------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'package_name', u'device_connection', u'device_language', u'developer', u'supported_devices', 'has_in_app_purchases', u'n_vungle_installs']
------interaction-----
Log Loss (Train): 0.405387
AUC (Train): 0.777427
Log Loss (Test): 0.407719
AUC (Test): 0.774127

------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'cloudux_template_id', u'city', u'device_platform', u'price_currency_code', u'platform', u'n_campaign_views', u'time_of_last_delivery_this_campaign']
------interaction-----
Log Loss (Train): 0.402476
AUC (Train): 0.782058
Log Loss (Test): 0.404968
AUC (Test): 0.778481

------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'creative_id', u'device_os_version', u'device_language', u'price_currency_code', u'platform', u'time_of_last_delivery_this_campaign', 'has_in_app_purchases']
------interaction-----
Log Loss (Train): 0.405729
AUC (Train): 0.776576
Log Loss (Test): 0.408404
AUC (Test): 0.772712

u'device_language', u'price_currency_code', u'content_rating', u'primary_category', u'device_platform', u'city', u'device_connection',u'min_os_version', u'time_zone', u'supported_devices'

------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'primary_category', u'device_platform', u'device_language', u'price_currency_code', u'content_rating', u'n_campaign_views', u'n_vungle_installs']
------interaction-----
[('advertiser_app_store_id', u'primary_category', u'device_language', u'content_rating'), ('advertiser_app_store_id', 'country_code', u'primary_category', u'device_platform', u'device_language', u'n_campaign_views'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_platform', u'device_language', u'price_currency_code', u'n_campaign_views', u'n_vungle_installs'), (u'primary_category', u'price_currency_code', u'content_rating', u'n_vungle_installs'), ('advertiser_app_store_id', 'country_code', u'device_platform', u'device_language', u'n_campaign_views', u'n_vungle_installs'), ('country_code', 'publisher_app_store_id', u'primary_category', u'device_language', u'price_currency_code', u'n_campaign_views'), ('advertiser_app_store_id', 'publisher_app_store_id', u'device_language', u'price_currency_code', u'n_vungle_installs'), ('advertiser_app_store_id', u'primary_category', u'device_platform', u'device_language', u'price_currency_code', u'content_rating'), ('advertiser_app_store_id', 'publisher_app_store_id', u'device_platform', u'device_language', u'price_currency_code', u'content_rating'), ('publisher_app_store_id', u'device_platform', u'device_language', u'content_rating', u'n_vungle_installs'), ('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'), ('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', 'country_code'), ('publisher_app_store_id', 'country_code'), (u'device_platform', u'device_language'), (u'price_currency_code', u'content_rating'), (u'n_campaign_views', u'n_vungle_installs')]
Train Regression Model
Log Loss (Train): 0.397312
AUC (Train): 0.789855
Log Loss (Test): 0.399423
AUC (Test): 0.786957
------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'primary_category', u'city', u'device_connection', u'platform', u'min_os_version', u'n_vungle_installs', u'n_campaign_views']
------interaction-----
[('country_code', u'device_connection', u'platform'), ('country_code', 'publisher_app_store_id', u'primary_category', u'platform', u'min_os_version', u'n_vungle_installs'), ('publisher_app_store_id', u'primary_category', u'city', u'min_os_version', u'n_vungle_installs', u'n_campaign_views'), ('publisher_app_store_id', u'city', u'device_connection', u'platform'), ('country_code', u'primary_category', u'city', u'device_connection', u'platform'), (u'primary_category', u'city', u'platform', u'min_os_version', u'n_campaign_views'), ('advertiser_app_store_id', 'country_code', u'device_connection', u'n_vungle_installs'), ('publisher_app_store_id', u'primary_category', u'city', u'min_os_version'), ('publisher_app_store_id', u'device_connection', u'platform', u'n_campaign_views'), ('advertiser_app_store_id', 'publisher_app_store_id', u'primary_category', u'city', u'device_connection', u'platform', u'min_os_version', u'n_vungle_installs'), ('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'), ('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', 'country_code'), ('publisher_app_store_id', 'country_code'), (u'city', u'device_connection'), (u'platform', u'min_os_version'), (u'n_vungle_installs', u'n_campaign_views')]
Train Regression Model
Log Loss (Train): 0.396940
AUC (Train): 0.790451
Log Loss (Test): 0.399244
AUC (Test): 0.787345



------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'creative_id', u'time_zone', u'city', 'primary_category', u'supported_devices', u'n_vungle_installs', u'n_campaign_views']
------interaction-----
[('country_code', 'publisher_app_store_id', u'time_zone', u'city', 'primary_category', u'supported_devices'), (u'city', 'primary_category'), ('advertiser_app_store_id', 'country_code', u'creative_id', u'time_zone', u'city', 'primary_category', u'supported_devices'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'time_zone', 'primary_category', u'supported_devices', u'n_vungle_installs', u'n_campaign_views'), ('advertiser_app_store_id', u'city', 'primary_category', u'supported_devices', u'n_vungle_installs', u'n_campaign_views'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'creative_id', u'city', 'primary_category', u'supported_devices', u'n_vungle_installs', u'n_campaign_views'), (u'time_zone', 'primary_category', u'n_vungle_installs'), ('advertiser_app_store_id', 'publisher_app_store_id', u'creative_id', u'time_zone', 'primary_category', u'supported_devices', u'n_vungle_installs'), ('publisher_app_store_id', u'time_zone', u'city', u'supported_devices', u'n_campaign_views'), ('advertiser_app_store_id', 'publisher_app_store_id', u'time_zone', 'primary_category', u'supported_devices'), ('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'), ('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', 'country_code'), ('publisher_app_store_id', 'country_code'), (u'time_zone', u'city'), ('primary_category', u'supported_devices'), (u'n_vungle_installs', u'n_campaign_views')]
Train Regression Model
Log Loss (Train): 0.397917
AUC (Train): 0.788977
Log Loss (Test): 0.400062
AUC (Test): 0.786129

------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'has_in_app_purchases', u'device_model', u'device_os_version', u'supported_devices', u'platform', u'n_campaign_views', u'n_vungle_installs']
------interaction-----
[(u'device_model', u'device_os_version', u'supported_devices', u'platform', u'n_vungle_installs'), ('country_code', 'publisher_app_store_id', u'device_model', u'device_os_version', u'n_campaign_views', u'n_vungle_installs'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'supported_devices', u'n_campaign_views'), ('advertiser_app_store_id', u'has_in_app_purchases', u'device_os_version', u'supported_devices', u'n_campaign_views', u'n_vungle_installs'), ('country_code', 'publisher_app_store_id', u'platform', u'n_campaign_views', u'n_vungle_installs'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_model', u'device_os_version', u'supported_devices', u'n_vungle_installs'), (u'has_in_app_purchases', u'device_model', u'n_campaign_views'), ('advertiser_app_store_id', 'publisher_app_store_id', u'has_in_app_purchases', u'device_model', u'supported_devices', u'platform', u'n_campaign_views', u'n_vungle_installs'), ('publisher_app_store_id', u'device_model', u'device_os_version', u'platform'), ('advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'device_os_version', u'n_vungle_installs'), ('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'), ('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', 'country_code'), ('publisher_app_store_id', 'country_code'), (u'device_model', u'device_os_version'), (u'supported_devices', u'platform'), (u'n_campaign_views', u'n_vungle_installs')]
Train Regression Model
Log Loss (Train): 0.396747
AUC (Train): 0.790555
Log Loss (Test): 0.399107
AUC (Test): 0.787199


In [ ]:

yX = yX_train[['is_install']+numerical_list+string_list].copy()


In [28]:
#clean data
b = preprocessing_data(yX,numerical_list,string_list)

df copied
filled na
converted to unknown


In [29]:
b.isnull().any()

is_install                                 False
time_delta_updated                         False
device_screen_width                        False
n_user_ratings                             False
price_value                                False
n_vungle_installs                          False
n_campaign_views                           False
n_min_downloads                            False
delivery_frequency_cap                     False
advertiser_app_store_id                    False
country_code                               False
publisher_app_store_id                     False
time_of_last_delivery_any_installed_app    False
is_publisher                               False
is_streaming                               False
dtype: bool

In [47]:
def logistic_model(X_train, y_train):
    logistic = LogisticRegression(C=1.0,penalty='l1')
    #svd = TruncatedSVD()
    #X_reduced = svd.fit_transform(X_train)
    #print X_reduced.shape
    #pipe = Pipeline(steps=[('svd', svd), ('logistic', logistic)])
    #n_components = [64,300,500,1000]
    #Cs = np.logspace(-4, 4, 3)

    #Parameters of pipelines can be set using ‘__’ separated parameter names:

    estimator = GridSearchCV(pipe,
                             dict(svd__n_components=n_components),
                            scoring='log_loss')
    estimator.fit(X_train, y_train)
    return estimator

In [24]:
numerical_list


['time_delta_updated',
 'device_screen_width',
 'n_user_ratings',
 'price_value',
 'n_vungle_installs',
 'n_campaign_views',
 'n_min_downloads',
 'delivery_frequency_cap']

In [25]:
string_list

['advertiser_app_store_id',
 'country_code',
 'publisher_app_store_id',
 'time_of_last_delivery_any_installed_app',
 'device_connection',
 'is_publisher',
 'is_streaming',
 'cloudux_template_id']

In [20]:

cont_features_pc = numerical_list
cat_interactions_pc = [     ('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'),
     ('advertiser_app_store_id', 'publisher_app_store_id'),
     ('advertiser_app_store_id', 'country_code'),
     ('publisher_app_store_id', 'country_code')]
cat_features_pc =string_list
other=[]
y=['is_install']
train, test = train_test_split(b, test_size=0.2,random_state= 42)  # splitting Train into Train/Test sets
X_train, y_train, X_test, y_test = train, train.pop('is_install'), test, test.pop('is_install')
    
feature_creator = FeatureCreator()
design_matrix_transformer = FeatureHasher(18, cat_features_pc, cont_features_pc, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

X_test = feature_creator.transform(X_test, inplace=True)
X_test, f_map = design_matrix_transformer.fit_transform(X_test)

X_train = feature_creator.transform(X_train, inplace=True)
X_train, f_map = design_matrix_transformer.fit_transform(X_train)

 

In [18]:
X_train.shape

(2213785, 262144)

In [21]:
  
logistic_baseline = logistic_model(X_train, y_train)

# Calculate prediction/probability of train and test
X_train_predictions = logistic_baseline.predict(X_train)
X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]

X_test_predictions = logistic_baseline.predict(X_test)
X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]


# Calculate metrics of train, validation and test set.
lr_ll_val = -logistic_baseline.best_score_

lr_ll_train = log_loss(y_train, X_train_predprob)
lr_auc_train = roc_auc_score(y_train, X_train_predprob)

lr_ll_test = log_loss(y_test, X_test_predprob)    
lr_auc_test = roc_auc_score(y_test, X_test_predprob)
    
# Print out the results
print "Best parameter: ", logistic_baseline.best_params_
     
print "Log Loss (Validation): %f" % lr_ll_val

print "Log Loss (Train): %f" % lr_ll_train
print "AUC (Train): %f" % lr_auc_train
#AUC[i]=lr_ll_test
#Log_lost[i]=lr_auc_test
print 'Log Loss (Test): %f' % lr_ll_test
print 'AUC (Test): %f' % lr_auc_test


Train Regression Model


/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127

Best parameter:  {'C': 0.05}
Log Loss (Validation): -0.404553
Log Loss (Train): 0.400862
AUC (Train): 0.786799
Log Loss (Test): 0.401010
AUC (Test): 0.785388
